In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [7]:
class TrafficSignClassification:
    def __init__(self, train_folder, test_folder, img_size=(32, 32), max_images_per_class=100):
        self.train_folder = train_folder
        self.test_folder = test_folder
        self.img_size = img_size
        self.max_images_per_class = max_images_per_class
        self.data = []
        self.labels = []
        self.cnn_model_path = "cnn_model.h5"  # Path to save/load CNN model

    def load_data(self, folder):
        data = []
        labels = []

        for label in range(43):  # 43 classes (0 to 42)
            class_folder = os.path.join(folder, str(label))
            if not os.path.exists(class_folder):
                print(f"Folder {class_folder} does not exist!")
                continue

            files = os.listdir(class_folder)
            for idx, file in enumerate(files):
                if idx >= self.max_images_per_class:
                    break
                file_path = os.path.join(class_folder, file)
                image = cv2.imread(file_path)
                if image is None:
                    continue
                image = cv2.resize(image, self.img_size)
                data.append(image)
                labels.append(label)

        data = np.array(data, dtype=np.float32) / 255.0
        labels = np.array(labels)
        return data, labels

    def prepare_data(self):
        print("Loading training data...")
        self.data, self.labels = self.load_data(self.train_folder)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.data, self.labels, test_size=0.2, random_state=42)
        
        print(f"Training data: {self.X_train.shape}")
        print(f"Test data: {self.X_test.shape}")

    def grid_search(self, model, param_grid):
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
        X_train_flat = self.X_train.reshape(self.X_train.shape[0], -1)  # Flatten Train images
        X_test_flat = self.X_test.reshape(self.X_test.shape[0], -1)  # Also Flatten Test images

        grid_search.fit(X_train_flat, self.y_train)
        best_model = grid_search.best_estimator_

        print(f"Best parameters for {model.__class__.__name__}: {grid_search.best_params_}")
        
        predictions = best_model.predict(X_test_flat)
        print(f"Classification Report:\n{classification_report(self.y_test, predictions)}")
        print(f"Confusion Matrix:\n{confusion_matrix(self.y_test, predictions)}")

        return best_model

    def train_svm(self):
        param_grid = {
            'C': [0.1, 1, 10],
            'kernel': ['rbf'],
            'gamma': ['scale', 'auto']
        }
        svm = SVC()
        best_svm_model = self.grid_search(svm, param_grid)
        
        # Save the SVM model after training
        with open('svm_model.pkl', 'wb') as f:
            pickle.dump(best_svm_model, f)
        print("SVM model saved as svm_model.pkl")
        
        return best_svm_model

    def train_knn(self):
        param_grid = {
            'n_neighbors': [3, 5, 7, 9]
        }
        knn = KNeighborsClassifier()
        best_knn_model = self.grid_search(knn, param_grid)
        
        # Save the KNN model after training
        with open('knn_model.pkl', 'wb') as f:
            pickle.dump(best_knn_model, f)
        print("KNN model saved as knn_model.pkl")
        return best_knn_model

    def train_decision_tree(self):
        param_grid = {
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        decision_tree = DecisionTreeClassifier()
        best_tree_model = self.grid_search(decision_tree, param_grid)
        
        # Save the Decision Tree model after training
        with open('decision_tree_model.pkl', 'wb') as f:
            pickle.dump(best_tree_model, f)
        print("Decision Tree model saved as decision_tree_model.pkl")
        return best_tree_model

    def train_random_forest(self):
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        random_forest = RandomForestClassifier()
        best_forest_model = self.grid_search(random_forest, param_grid)
        
        # Save the Random Forest model after training
        with open('random_forest_model.pkl', 'wb') as f:
            pickle.dump(best_forest_model, f)
        print("Random Forest  model saved as random_forest_model.pkl")
        return best_forest_model

    def train_softmax_regression(self):
        param_grid = {
            'C': [0.1, 1, 10],
            'max_iter': [100, 200, 300]
        }
        logistic = LogisticRegression(multi_class='multinomial', max_iter=1000)
        best_softmax_model = self.grid_search(logistic, param_grid)
        
        # Save the Softmax Regression model after training
        with open('softmax_model.pkl', 'wb') as f:
            pickle.dump(best_softmax_model, f)
        print("Softmax Regression model saved as softmax_model.pkl")
        return best_softmax_model

    def train_cnn(self):
        y_train_categorical = to_categorical(self.y_train, 43)
        y_test_categorical = to_categorical(self.y_test, 43)

        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(self.img_size[0], self.img_size[1], 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(43, activation='softmax')
        ])
        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(self.X_train, y_train_categorical,
                            validation_data=(self.X_test, y_test_categorical),
                            epochs=10, batch_size=32, verbose=1)

        _, accuracy = model.evaluate(self.X_test, y_test_categorical, verbose=0)
        print(f"CNN Accuracy: {accuracy * 100:.2f}%")
        
        # Save the CNN model to a file
        model.save(self.cnn_model_path)
        print(f"CNN model saved as {self.cnn_model_path}")
        self.cnn_model = model
        
    def load_cnn_model(self):
        if os.path.exists(self.cnn_model_path):
            self.cnn_model = load_model(self.cnn_model_path)
            print(f"CNN model loaded from {self.cnn_model_path}")
        else:
            print(f"CNN model file {self.cnn_model_path} does not exist. Train the model first.")

    def visualize_samples(self):
        plt.figure(figsize=(12, 8))
        for i in range(9):
            plt.subplot(3, 3, i + 1)
            plt.imshow(self.X_train[i])
            plt.title(f"Label: {self.y_train[i]}")
            plt.axis("off")
        plt.tight_layout()
        plt.show()

In [9]:
# Paths of train and test data folders
csv_train = "train.csv"
csv_test = "test.csv"
test_folder = r"E:\DataScience BootCamp\Ayad_Collection\Udemy\Projects\Computer Vision\Computetr Vision\Traffic Classification\Test"
train_folder = r"E:\DataScience BootCamp\Ayad_Collection\Udemy\Projects\Computer Vision\Computetr Vision\Traffic Classification\Train"


# Initialize the TrafficSignClassification object
classifier = TrafficSignClassification(train_folder, test_folder)

# Prepare the data
classifier.prepare_data()

# Train and evaluate the SVM model
print("Training SVM model...")
svm_model = classifier.train_svm()

print("Training KNN model...")
knn_model = classifier.train_knn()

print("Training Decision Tree model...")
tree_model = classifier.train_decision_tree()

print("Training Random Forest model...")
forest_model = classifier.train_random_forest()

print("Training CNN model...")
cnn_model = classifier.train_cnn()

print("Training Softmax Regression model...")
softmax_model = classifier.train_softmax_regression()

classifier.visualize_samples()

Loading training data...
Training data: (3440, 32, 32, 3)
Test data: (860, 32, 32, 3)
Training SVM model...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best parameters for SVC: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      1.00      1.00        24
           2       1.00      1.00      1.00        25
           3       1.00      1.00      1.00        18
           4       1.00      0.81      0.89        21
           5       1.00      1.00      1.00        21
           6       1.00      0.94      0.97        18
           7       1.00      0.95      0.97        20
           8       0.88      1.00      0.94        22
           9       1.00      1.00      1.00        13
          10       1.00      1.00      1.00        25
          11       0.86      0.96      0.91        25
          12       0.90      1.00   

Best parameters for RandomForestClassifier: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      0.96      0.98        24
           2       0.96      1.00      0.98        25
           3       0.86      1.00      0.92        18
           4       1.00      1.00      1.00        21
           5       1.00      1.00      1.00        21
           6       1.00      0.94      0.97        18
           7       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        22
           9       1.00      1.00      1.00        13
          10       1.00      1.00      1.00        25
          11       1.00      0.92      0.96        25
          12       1.00      1.00      1.00        26
          13       0.90      1.00      0.95        18
          14       1.00      1.00 

C:\Users\mr\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.0510 - loss: 3.6860 - val_accuracy: 0.4302 - val_loss: 2.6199
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3473 - loss: 2.4820 - val_accuracy: 0.7279 - val_loss: 1.1968
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6082 - loss: 1.3878 - val_accuracy: 0.8616 - val_loss: 0.6580
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7239 - loss: 0.9306 - val_accuracy: 0.9279 - val_loss: 0.3935
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8090 - loss: 0.6527 - val_accuracy: 0.9512 - val_loss: 0.2857
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8520 - loss: 0.4973 - val_accuracy: 0.9698 - val_loss: 0.1790
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8959 - loss: 0.3676 - val_accuracy: 0.9826 - val_loss: 0.1216
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9041 - loss: 0.3181 - val_accuracy: 0.980

CNN Accuracy: 98.60%


ImportError: `save_model()` using h5 format requires h5py. Could not import h5py.

In [10]:
with open('knn_model.pkl', 'rb') as f:
    knn_model = pickle.load(f)

In [11]:
with open('random_forest_model.pkl', 'rb') as f:
    random_forest_model = pickle.load(f)

In [12]:
with open('decision_tree_model.pkl', 'rb') as f:
    decision_tree_model = pickle.load(f)

In [15]:
with open('softmax_model.pkl', 'rb') as f:
    softmax_model = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'softmax_model.pkl'

In [14]:
with open('svm_model.pkl', 'rb') as f:
    svm_model = pickle.load(f)